# AB2017 - Python ile Pratik Makine Öğrenimi

Bu dökümanda çok basit düzeyde zararlı yazılım sınıflandırması yapan bir sistem eğiteceğiz. Örneğin orjinalini [şu adreste](https://www.youtube.com/watch?v=iLNHVwSu9EA&t=2s) bulabilirsiniz. Gerekli bazı importları yaparak başlıyoruz:

In [3]:
import pandas as pd
import numpy as np
import pickle
from sklearn import ensemble
from sklearn import model_selection, tree, linear_model
from sklearn.feature_selection import SelectFromModel
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

Sistem için gerekli verileri dosyadan yüklüyoruz. Zararlı yazılım örnekleri için isim ve bütünlük koduna ihtiyacımız olmadığından **Name** ve **md5** sütunlarını almıyoruz.

In [6]:
data = pd.read_csv("data/data.csv", sep="|")
X = data.drop(["Name", "md5", "legitimate"], axis=1).values
y = data['legitimate'].values

Başlamadan önce öznitelikler üstünde dönüşüm işlemi yapıyoruz. Bunun için ayrıca bir sınıflandırıcı model gerekiyor. Ardından tipik **train_test_split** metodu ile %30 test için olacak şekilde verisetini bölümlüyoruz.

In [9]:
feature_selector = ensemble.ExtraTreesClassifier().fit(X, y)
model = SelectFromModel(feature_selector, prefit=True)
X_new = model.transform(X)
feature_count = X_new.shape[1]

X_train, X_test, y_train, y_test = model_selection.train_test_split(X_new, y, test_size=0.30)

Bir önceki örneklerden farklı olarak bu sefer birden fazla modeli ve parametre kombinasyonlarını deneyeceğiz. Bunun için yapılabilecek en basit şey anahtar elemanı model, değeri parametre sözlüğü olan bir sözlük oluşturmak. Buradan elde ettiğimiz model ve sonuçları ise **model_results** adlı sözlükte tutacağız.

**UYARI:** Bu işlem bilgisayarınızın işlem gücüne bağlı olarak cidden uzun sürebilir. Eğer hemen sonuç almak istiyorsanız, eğitmeden önce modellerin bazılarını silin veya yorum satırına alın.

In [27]:
model_results = {}

models_params = {
    GaussianNB(): {},

    tree.DecisionTreeClassifier(): {
        'max_depth': list(range(1, 20)),
        'max_features': list(range(1, feature_count + 1))
    },
    
    MLPClassifier(): {
        'activation': ['tanh', 'relu', 'logistic'],
        'solver': ['sgd', 'adam'],
        'alpha': [1e-4, 1e-3, 1e-2],
        'max_iter': [200, 500, 1000]
    },
    
    ensemble.RandomForestClassifier(): {
        'n_estimators': [10, 100, 1000],
        'max_depth': list(range(1, 20)),
        'max_features': list(range(1, feature_count + 1))
    }
}

Burada **models_params** adlı sözlüğün içindeki her bir eleman çifti için **GridSearchCV** kullanıyoruz. **n_jobs** parametresi ile paralelde kaç tane eğitim yapılabileceğini ayarlayabiliriz. Bunun genellikle işlemci çekirdek veya izlek sayısına eşit olması idealdir.

Ardından her bir model için test skoru çıkartıp bunu **model_results** adlı sözlükte, anahtarı model olacak şekilde tutuyoruz.

In [15]:
from sklearn.grid_search import GridSearchCV

for model, params in models_params.iteritems():
    grid = GridSearchCV(estimator=model, param_grid=params, n_jobs=4)
    grid.fit(X_train, y_train)
    
    test_score = grid.best_estimator_.score(X_test, y_test)
    model_results[grid.best_estimator_] = test_score

Test skoru en yüksek olan modeli doğrudan seçip kaydedebiliriz.

In [16]:
best_model = max(model_results, key=model_results.get)

print best_model
print model_results[best_model]

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=14,
            max_features=14, max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
0.991066038875


Modeli kaydetmek için bu **pickle** kullanalım. **pickle** ve **cPickle** modülleri çoğu Python dağıtımında yüklü olarak gelmektedir.

In [18]:
import pickle

with open('data/malware_detector.pkl', 'wb') as f:
    pickle.dump(best_model, f)
    

Modeli tekrardan dosyadan yükleyip kullanmaya başlayabiliriz:

In [26]:
with open('data/malware_detector.pkl', 'rb') as f:
    model = pickle.load(f)
    
sample = X_test[550, :].reshape(1, -1)

print(u'Tahmin edilen sınıf: %d' % model.predict(sample)[0])
print(u'Confidence değerleri: %s' % model.predict_proba(sample)[0])

Tahmin edilen sınıf: 0
Confidence değerleri: [  9.99779978e-01   2.20022002e-04]
